In [3]:
from omegaconf import DictConfig, OmegaConf
import inspect

import shutil
import numpy as np

from pathlib import Path
from dataclasses import asdict
from carps.utils.task import Task

from optbench import functions
from optbench.abstract_function import AbstractFunction

def fullname(o):
    try:
        # if o is a class or function, get module directly
        module = o.__module__
    except AttributeError:
        # then get module from o's class
        module = o.__class__.__module__
    try:
        # if o is a class or function, get name directly
        name = o.__qualname__
    except AttributeError:
        # then get o's class name
        name = o.__class__.__qualname__
    # if o is a method of builtin class, then module will be None
    if module == 'builtins' or module is None:
        return name
    return module + '.' + name

base_path = Path("../optbench/configs/problem")

# Problem ids
benchmark_id = "OptBench"

# from YAHPO paper
def get_n_trials(dimension: int) -> int:
    return int(np.ceil(20 + 40 * np.sqrt(dimension)))

######################

# Generate configs for synthetic functions

target_path = base_path / benchmark_id
if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "optbench.optbench_problem.OptBenchProblem"

function_classes = [
    c[1]
    for c in inspect.getmembers(functions, inspect.isclass)
    if issubclass(c[1], AbstractFunction) and c[1] != AbstractFunction
]
dimensions = [2, 5, 10]

identifiers = []
for function_class in function_classes:
    for dim in dimensions:
        problem_id = f"{function_class.__name__}-{dim}"
        task = Task(
            dimensions=dim,
            n_trials=get_n_trials(dim),
            time_budget=None,
            n_workers=1,
            n_objectives=1,
            objectives=["quality"],
            is_multifidelity=False,
            fidelity_type=None,
            min_budget=None,
            max_budget=None,
            has_constraints=False,
            domain="synthetic",
            objective_function_approximation="real",
            has_virtual_time=False,
            deterministic=True,
            search_space_n_categoricals=0,
            search_space_n_ordinals=0,
            search_space_n_integers=0,
            search_space_n_floats=dim,
            search_space_has_conditionals=False,
            search_space_has_forbiddens=False,
            search_space_has_priors=False
        )
        cfg = DictConfig(
            {
                "benchmark_id": benchmark_id,
                "problem_id": problem_id,
                "problem": {
                    "_target_": problem_class,
                    "function": {
                        "_target_": fullname(function_class),
                        "dim": dim,
                        "seed": "${seed}",
                    },
                },
                "task": asdict(task)
            }
        )
        identifier = f"{function_class.__name__}_{dim}"
        identifiers.append(identifier)
        fn = target_path / (identifier + ".yaml")
        yaml_str = OmegaConf.to_yaml(cfg=cfg)
        yaml_str = "# @package _global_\n" + yaml_str
        fn.write_text(yaml_str)
        print(cfg)

{'benchmark_id': 'OptBench', 'problem_id': 'Ackley-2', 'problem': {'_target_': 'optbench.optbench_problem.OptBenchProblem', 'function': {'_target_': 'optbench.functions.Ackley', 'dim': 2, 'seed': '${seed}'}}, 'task': {'n_trials': 77, 'time_budget': None, 'n_workers': 1, 'n_objectives': 1, 'objectives': ['quality'], 'is_multifidelity': False, 'fidelity_type': None, 'min_budget': None, 'max_budget': None, 'has_constraints': False, 'domain': 'synthetic', 'objective_function_approximation': 'real', 'has_virtual_time': False, 'deterministic': True, 'dimensions': 2, 'search_space_n_categoricals': 0, 'search_space_n_ordinals': 0, 'search_space_n_integers': 0, 'search_space_n_floats': 2, 'search_space_has_conditionals': False, 'search_space_has_forbiddens': False, 'search_space_has_priors': False}}
{'benchmark_id': 'OptBench', 'problem_id': 'Ackley-5', 'problem': {'_target_': 'optbench.optbench_problem.OptBenchProblem', 'function': {'_target_': 'optbench.functions.Ackley', 'dim': 5, 'seed': '$

In [6]:
",".join(identifiers)

'synth_Ackley_2,synth_Ackley_5,synth_Ackley_10,synth_Levy_2,synth_Levy_5,synth_Levy_10,synth_Schwefel_2,synth_Schwefel_5,synth_Schwefel_10'